In [1]:
%load_ext autoreload

In [1]:
%autoreload 2

UsageError: Line magic function `%autoreload` not found.


In [2]:
import bittensor
import os
import torch
import torch.multiprocessing as mp 
import time
from loguru import logger
from termcolor import colored
import nest_asyncio 
nest_asyncio.apply()
logger.remove()

# **Init Bittensor** <a class="anchor" id="Init-Bittensor"></a>

## **Wallet** <a class="anchor" id="Wallet"></a>

In [3]:
# WALLET: Holds keys to run your miner and control funds.

# *****
# IMPORTANT: Store the mnemonic for **both** your hot and coldkey 
# you will need these to recover your keys if you forget your password or lose access to this machine.
# ******

# Fill in below to name your wallet and keys.
YOUR_WALLET_NAME = 'colab'
YOUR_HOTKEY_NAME = 'colab_hot'

# Fill in below if your need to regenerate your keys.
use_mnemonic = False # Set to true for key regeneration.
coldkey_mnemonic = "<to be filled>".split(' ')
hotkey_mnemonic = "<to be filled>".split(' ')

# Create the wallet object.
wallet = bittensor.Wallet(
    path = "~/.bittensor/wallets/",
    name = YOUR_WALLET_NAME,
    hotkey = YOUR_HOTKEY_NAME
)

# Optionally regens/creates your wallet keys.
if not wallet.has_coldkeypub:
    if use_mnemonic:
        wallet.regenerate_coldkey(mnemonic = coldkey_mnemonic, use_password = True)
    else:
        wallet.create_new_coldkey(n_words = 12, use_password = True )
if not wallet.has_hotkey:
    if use_mnemonic:
        wallet.regenerate_hotkey(mnemonic = hotkey_mnemonic)
    else:
        wallet.create_new_hotkey(n_words = 12)

# Assert before continuing
assert wallet.has_hotkey
assert wallet.has_coldkeypub

Loaded coldkey.pub: 0x3c9cd1679888e5660b0c8e4b8a17a1719c0cb7f05b5c624a856b421b52290515
Loaded hotkey: 0x80cacfbdf7b155b39de22680a7cb14c61a8f95df702c92f5f142d25cca37c545


## **Subtensor**


In [4]:
subtensor = bittensor.Subtensor(
    subtensor_network = 'kusanagi'
)
print (bittensor.Config.toString(subtensor.config))


subtensor:
  chain_endpoint: null
  network: kusanagi
wallet:
  hotkey: default
  name: default
  path: ~/.bittensor/wallets/



## **Metagraph** <a class="anchor" id="Metagraph"></a> 


In [5]:
subtensor.connect()

Successfully connected to kusanagi endpoint: 157.230.11.31:9944


True

In [6]:
metagraph = bittensor.Metagraph(
    subtensor = subtensor,
)

In [7]:
metagraph.sync()

.........................................................................................................................................................................................................................................................................................................................................................................................................................................................................



In [8]:
chain_block = metagraph.block()
uids_on_chain = metagraph.uids()
n_neurons = torch.max( metagraph.uids() )
n_online = torch.numel(torch.where( metagraph.block() - metagraph.lastemit() < 1000 )[0])
print ('The chain is at block: {}\n'.format(metagraph.block()))
print ('There are {} subscribed miner neurons \n'.format(n_neurons))
print ('These are their uids: \n{}\n'.format(metagraph.uids()))
print ('{} have set weights in the last 1000 blocks\n'.format(n_online))
print ('\u03C4{} is staked'.format(torch.sum(metagraph.S())))

The chain is at block: 1118538

There are 456 subscribed miner neurons 

These are their uids: 
tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
         42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
         56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
         84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
         98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
        112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
        126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
        140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
        154, 155, 156, 157, 158, 159, 160, 161

## **Dendrite**

In [9]:
dendrite = bittensor.Dendrite(
    wallet = wallet,
    dendrite_multiprocess = False,
    receptor_do_backoff = False,
    receptor_timeout = 10,
)
print (bittensor.Config.toString(dendrite.config))



dendrite:
  debug: false
  multiprocess: false
receptor:
  do_backoff: false
  max_backoff: 100
  pass_gradients: true
  timeout: 10
wallet:
  hotkey: colab_hot
  name: colab
  path: ~/.bittensor/wallets/



# **Nucleus**  <a class="anchor" id="Nucleus"></a>
**a.k.a your unique machine learning model**

## **GPT2-Nucleus** <a class="anchor" id="GPT2-Nucleus"></a>

In [10]:
import torch.nn as nn
import torch.nn.functional as F
from types import SimpleNamespace
from transformers import GPT2Config, GPT2Model

class GPT2Nucleus(torch.nn.Module):
    """ A simple-as-it-gets bittensor nucleus using a GPT2 kernel
    """
    def __init__(self):
        super().__init__()
        huggingface_config = GPT2Config( vocab_size=bittensor.__vocab_size__, n_embd=bittensor.__network_dim__, n_layer=2, n_head=1, n_inner=8 )
        self.transformer = GPT2Model(huggingface_config)
        self.hidden_layer = nn.Linear( bittensor.__network_dim__, bittensor.__network_dim__ )
        self.target_layer = nn.Linear( bittensor.__network_dim__, bittensor.__vocab_size__, bias=False )
        self.loss_fct = nn.CrossEntropyLoss()
        
        # The scores you learn for other neurons in the network.
        self.row_weights = torch.ones([1], requires_grad=True)
        
        # Bittensor components that need to be set before you call remote_forward.
        self.metagraph = None
        self.dendrite = None
    
    
    def local_forward(self, inputs: torch.LongTensor) -> SimpleNamespace:
        """ Applies a forward pass to the model **without** queries to the network.
        """
        # To be filled.
        output = SimpleNamespace()

        # Apply our GPT transformer model.
        # local_context.shape = [ batch_size, sequence_len, network_dim ]
        output.local_context = self.transformer(input_ids=inputs, return_dict=True).last_hidden_state

        # Apply our dense layer and project it onto our hidden layer.
        # local_hidden.shape = [ batch_size, sequence_len, network_dim ]
        output.local_hidden = self.hidden_layer( output.local_context )

        # Project to our target dimension.
        # local_targets.shape = [ batch_size, sequence_len, vocab_size ]
        output.local_targets = self.target_layer( output.local_hidden )

        # Compute LM-loss 
        shift_targets = output.local_targets[..., :-1, :].contiguous()
        shift_inputs = inputs[..., 1:].contiguous()
        output.local_loss = self.loss_fct(shift_targets.view(-1, shift_targets.size(-1)), shift_inputs.view(-1))

        return output

    def remote_forward( self, inputs: torch.LongTensor, n_to_query:int = 10) -> SimpleNamespace:
        """ Applies a forward pass to the model **with** queries to the network.
        """
        # Sanity checks.
        assert self.metagraph != None, 'you must assign model.metagraph, before you run a remote call.'
        assert self.dendrite != None, 'you must assign model.dendrite, before you run a remote call.'
        
        # Run the local part of the model.
        output = self.local_forward( inputs )

        # Make queries to the network.
        output = self.filter_and_make_queries( output, inputs, n_to_query )
        
        # Compute the distillation loss between the local and remote context (produced by the network query.)
        output.distillation_loss = F.mse_loss( output.local_context, output.remote_context.detach() )

        # Apply the hidden dense layer to the context.
        # remote_hidden.shape = [ batch_size, sequence_len, network_dim ]
        output.remote_hidden = self.hidden_layer( output.remote_context )

        # Project onto our target dimension.
        # remote_hidden.shape = [ batch_size, sequence_len, vocab_size ]
        output.remote_targets = self.target_layer( output.remote_hidden )

        # Compute our loss against the remote context.
        shift_targets = output.remote_targets[..., :-1, :].contiguous()
        shift_inputs = inputs[..., 1:].contiguous()
        output.remote_loss = self.loss_fct(shift_targets.view(-1, shift_targets.size(-1)), shift_inputs.view(-1))

        return output
    
    def filter_and_make_queries(self, output: SimpleNamespace, inputs:torch.LongTensor, n_to_query:int = 10) -> SimpleNamespace:
        """ Filters peers based on activity and makes RPC queries.
        """
        
        # Pad the row weights to the network dimension.
        self.row_weights = torch.nn.functional.pad(
            self.row_weights, 
            pad = [0, self.metagraph.n() - self.row_weights.numel() ],
            value = torch.mean(self.row_weights).item() # New values at the mean.
        ).clone().detach().requires_grad_(True)
       
        # Get all neuron uids.
        # all_uids = [n]
        all_uids = metagraph.uids() 
        
        # Filter uids based on last emit.
        # filtered_uids = [ m <= n]
        filtered_uids = all_uids[ torch.where( metagraph.block() - metagraph.lastemit() < 1000 ) ] 

        # Get corresponding weights.
        # filtered_weights = [ m ]
        filtered_weights = self.row_weights[ filtered_uids ]

        # Get topk weights for filtered uids
        # query_indices = [ n_to_query <= m ]
        gamma = 0.9
        output.query_weights, indices = torch.topk(
            filtered_weights + torch.rand_like(filtered_weights) * gamma, 
            min(n_to_query, torch.numel(filtered_weights))
        )
        
        # Duplicate inputs for each request.
        # inputs_to_send = n_to_query * [ batch_size, sequence_length ]
        output.query_uids = filtered_uids[ indices.tolist() ]
        inputs_to_send = [ inputs for _ in output.query_uids.tolist() ]
        neurons_to_query = [ metagraph.neurons()[ i ] for i in output.query_uids.tolist() ]
        
        # Make network calls. 
        # responses = n_to_query * [batch_size, sequence_length, network_dimension]
        output.codes, responses = self.dendrite.forward_text( 
            neurons = neurons_to_query, 
            inputs = inputs_to_send
        )
        
        # Weight-join responses.
        # remote_context = [batch_size, sequence_length, network dimension]
        stacked_responses = torch.stack( responses, dim=2 )
        output.remote_context = torch.matmul( torch.transpose( stacked_responses, dim0=2, dim1=3), output.query_weights)

        return output

In [11]:
# Your nucleus
model = GPT2Nucleus()
model.metagraph = metagraph
model.dendrite = dendrite

## **Test-Nucleus** <a class="anchor" id="Test-Nucleus"></a>

In [13]:
# Test remote forward call.
inputs = torch.tensor([ 
                        bittensor.__tokenizer__()('the cat')['input_ids'], # Text sequence 1
                        bittensor.__tokenizer__()('the dog')['input_ids'],  # Text sequence 2 
                        bittensor.__tokenizer__()('the bird')['input_ids'],  # Text sequence 3 

                      ])
output = model.remote_forward( inputs, n_to_query = 50 )
loss = output.local_loss + output.remote_loss + output.distillation_loss
loss.backward() #
print ('The joined remote context from the network is: \n{}\nwith shape: {}\n'.format(output.remote_context, output.remote_context.shape))
print ('The distillation loss between your local and remote context is  {}\n'.format(output.distillation_loss))
print ('The loss with respect to your local context and the targets is {}\n'.format(output.local_loss))
print ('The loss with respect to your remote context and the targets is {}\n'.format(output.remote_loss))
print ('You queried {} remote neurons with uids\n {}\n\nand response codes\n {}\n'.format(torch.numel(output.codes), output.query_uids, output.codes.tolist()))
print ('with row weights:\n{}'.format(model.row_weights))
print ('and gradients w.r.t your row weights are\n', [float('{:0.3f}'.format(model.row_weights.grad[idx].item())) for idx in output.query_uids.tolist()])



The joined remote context from the network is: 
tensor([[[-13.8122,   3.5462,  13.2160,  ...,  -7.4968,  -8.9330,   8.3471],
         [-12.8419,  -3.2254,  16.0121,  ..., -10.4737,  -2.0244,   3.3301]],

        [[-14.7074,   4.0230,   9.9079,  ...,  -1.5730,  -3.8949,   6.0364],
         [-12.8490,   1.1805,  13.7353,  ...,  -7.2885,   0.7176,   3.9728]],

        [[-17.5738,  -2.5029,  14.9698,  ...,  -7.9361,  -3.0595,   1.1690],
         [-16.0426,   0.2690,  14.5040,  ...,  -6.0156,  -1.9508,   2.1865]]],
       grad_fn=<UnsafeViewBackward>)
with shape: torch.Size([3, 2, 512])

The distillation loss between your local and remote context is  128.0544891357422

The loss with respect to your local context and the targets is 10.822928428649902

The loss with respect to your remote context and the targets is 17.44337272644043

You queried 50 remote neurons with uids
 tensor([333, 223,  83, 189,  31, 320, 197, 211,  89, 199, 185, 214,  92, 196,
          2, 225, 207, 398, 186,  44,  52,

# **Training** <a class="anchor" id="Training"></a>

## **Training-Loop** <a class="anchor" id="Training-Loop"></a>

In [14]:
from torch.nn.utils import clip_grad_norm_
import torch.nn.functional as F
from loguru import logger
import random
from datasets import load_dataset
import time
import os

# ---- Dataset ---- 
dataset = load_dataset('ag_news')['train']
def nextbatch(data, batch_size, tokenizer):
    """ Returns a random batch of sentences from text dataset.
    """
    batch_text = []
    for _ in range(batch_size):
        batch_text.append(data[random.randint(0, len(data))]['text'])
    batch_inputs = tokenizer(batch_text, return_tensors='pt', padding=True, truncation=True)['input_ids']
    return batch_inputs

# --- Training Logger ----
logger.remove()
training_log_dir = os.path.expanduser('~/logs/training.log')
logger.add(training_log_dir, filter=lambda record: "training" in record["extra"], enqueue=True, backtrace=True, diagnose=True, rotation="500 MB")
def show_training_logs(length: int = 25):
    ! tail -n $length $training_log_dir

# ---- Tokenizer ----
# For encoding text inputs.
tokenizer = bittensor.__tokenizer__()

# ---- Optimizer ----
# For applying gradient steps to the local model.
optimizer = torch.optim.SGD( model.parameters(), lr = 0.1, momentum = 0.99 )

# ---- Training Loop -----
def train( 
        stop_training: mp.Event,
    ):
    # ---- Loop until event is set ----
    training_step = 0
    batch_size = 5
    logger.bind(training=True).info('Loop starting... ')
    while not stop_training.is_set():
        try:
            optimizer.zero_grad() # Zeros out gradients for next accummulation

            # ---- Forward pass ----
            inputs = nextbatch( dataset, batch_size, tokenizer )
            output = model.remote_forward( inputs, n_to_query=50 )

            # ---- Backward pass ----
            loss = output.local_loss + output.remote_loss + output.distillation_loss
            loss.backward() # Accumulates gradients on the model.
            clip_grad_norm_(model.parameters(), 0.8) # clip model gradients
            optimizer.step() # Applies accumulated gradients.

            # ---- Step logs ----
            logger.bind(training=True).info('->\nuids:{}\ncodes:{}\nweights:{}\ngrads:{}', 
                  output.query_uids, 
                  output.codes.tolist(), 
                  [float('{:0.3f}'.format(x)) for x in output.query_weights.tolist()],
                  [float('{:0.3f}'.format(model.row_weights.grad[idx].item())) for idx in output.query_uids.tolist()])      
            logger.bind(training=True).info('gs:{} loss(local):{} loss(remote):{} loss(distill):{} dendrite:{}',
                  colored('{}'.format( training_step ), 'red'),
                  colored('{:.4f}'.format(output.local_loss.item()), 'green'),
                  colored('{:.4f}'.format(output.remote_loss.item()), 'blue'),
                  colored('{:.4f}'.format(output.distillation_loss.item()), 'red'),
                  dendrite)

            # --- Train and normalize row weights ---
            model.row_weights = torch.clamp(model.row_weights - 0.001 * model.row_weights.grad, 0, 1)
            model.row_weights = F.normalize( model.row_weights, p = 1, dim = 0 ).clone().detach().requires_grad_(True)

            training_step += 1
        except Exception as e:
            logger.bind(training=True).exception("Training iteration exception.")
    logger.bind(training=True).complete()

Using custom data configuration default
Reusing dataset ag_news (/Users/const/.cache/huggingface/datasets/ag_news/default/0.0.0/fb5c5e74a110037311ef5e904583ce9f8b9fbc1354290f97b4929f01b3f48b1a)


## **Training Thread Runners** <a class="anchor" id="Training-Thread-Runners"></a>

In [15]:
import threading
import torch.multiprocessing as mp 
import sys

join_timeout = 10

if 'quit_training' in locals():
    quit_training.set()
if 'training_thread' in locals() and training_thread.is_alive():
    training_thread.join( timeout = join_timeout )

quit_training = mp.Event()
training_thread = threading.Thread( target = train, args = (quit_training,),  name = 'training', daemon=True)

def stop_training():
    global quit_training
    global training_thread
    quit_training.set()
    if not training_thread.is_alive():
        return
    logger.bind(training=True).info("Joining...")
    training_thread.join( timeout = join_timeout )
    if not training_thread.is_alive():
        print ('Joined training thread',)
        logger.bind(training=True).info('Joined.')
    else:
        print ('Failed to join training thread')

def start_training():
    global quit_training
    global training_thread
    stop_training()
    quit_training = mp.Event()
    training_thread = threading.Thread( target = train, args = (quit_training,), name = 'training', daemon=True)
    training_thread.start()
    logger.bind(training=True).info("Started training.")
    print('new training thread:', training_thread)

In [16]:
start_training()

new training thread: <Thread(training, started daemon 123145618812928)>


In [17]:
print (training_thread.is_alive())

True


In [18]:
stop_training()

Joined training thread


In [25]:
show_training_logs(20)

grads:[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
2021-04-06 10:32:30.482 | INFO     | __main__:train:68 - gs:3 loss(local):9.5550 loss(remote):10.8234 loss(distill):0.9995 dendrite:(0.005q/s|⤊ 1621.6/⤋ 0.4kB/s)
2021-04-06 10:32:38.370 | INFO     | __main__:train:62 - ->
uids:tensor([203,  34, 208, 229, 400, 172, 190, 217, 210,  15, 177, 398,  13,   0,
        121,  52,   5, 194,  75,  31, 396,  99, 197, 206,  51, 195, 193, 154,
        184, 399,  74, 182, 425, 227,  12, 116,  82, 189, 201, 198, 424,  53,
        225, 320,  21,  64, 341, 220,  88, 381])
codes:[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 21, 1, 1, 1, 1, 3, 3, 1, 1, 1, 1, 1, 1, 1, 21, 1, 1, 1, 1, 3, 1, 1, 3, 1, 1, 1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 3]
weights:[0.898, 0.898, 0.896, 0.89, 0.884, 0.869, 0.843, 0.84, 0.822, 0.81

In [26]:
torch.topk( model.row_weights, 10 )

torch.return_types.topk(
values=tensor([0.0022, 0.0022, 0.0022, 0.0022, 0.0022, 0.0022, 0.0022, 0.0022, 0.0022,
        0.0022], grad_fn=<TopkBackward>),
indices=tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]))

# **Serving** <a class="anchor" id="Serving"></a>

In [27]:
external_ip = bittensor.utils.networking.get_external_ip()
print (external_ip)

172.58.142.212


In [28]:
axon = bittensor.Axon(
    axon_local_ip = '127.0.0.1',
    axon_local_port = 8080,
    axon_external_port = 8080,
    axon_external_ip = bittensor.utils.networking.get_external_ip()
)
print (bittensor.Config.toString(axon.config))


axon:
  backward_processing_timeout: 5
  external_ip: 172.58.142.212
  external_port: 8080
  forward_processing_timeout: 5
  local_ip: 127.0.0.1
  local_port: 8080
  max_workers: 10
  use_upnpc: false
wallet:
  hotkey: default
  name: default
  path: ~/.bittensor/wallets/



In [29]:
# Start the axon serving endpoint.
axon.start()

start


## **Serving-Loop** <a class="anchor" id="Serving-Loop"></a>

In [30]:
# ---- Serving logger ----
serving_log_dir = os.path.expanduser('~/logs/serving.log')
logger.remove()
logger.add(serving_log_dir, filter=lambda record: "serving" in record["extra"], enqueue=True, backtrace=True, diagnose=True, rotation="500 MB")
def show_serving_logs(length: int = 25):
    ! tail -n $length $serving_log_dir

# ---- Serving loop -----
def serve ( 
  stop_serving: mp.Event,
):

    # ---- Loop until event is set -----
    serving_step = 0
    logger.bind(serving=True).info('Serving thread started: ')
    while not stop_serving.is_set():
        try:
            
            # ---- Pull request ----
            logger.bind(serving=True).info('Axon:{}, waiting for query ... ', axon)
            pong, pubkey, inputs, modality = axon.next_forward_item( timeout = 1.0 )
            logger.bind(serving=True).info('Recieved: {}', [ pong, pubkey, inputs, modality ])

            # ---- Process request ----
            if None not in [ pong, pubkey, inputs, modality ]:
                logger.bind(serving=True).info('Recieved Query: from:{}, inputs.shape:{}', pubkey, inputs.shape)
                output = model.local_forward( inputs ).local_hidden
                pong.send( output.detach() )
                logger.bind(serving=True).info('Sent response: to:{}, output.shape:{}', pubkey, output.shape)

        except Exception as e:
            logger.bind(serving=True).exception('Error in forward process with error {}', e)



## **Serving Thread Runners** <a class="anchor" id="Serving-Thread-Runners"></a>

In [31]:
join_timeout = 10

if 'quit_serving' in locals():
    quit_serving.set()
if 'serving_thread' in locals() and serving_thread.is_alive():
    serving_thread.join( timeout = join_timeout )

quit_serving = mp.Event()
serving_thread = threading.Thread( target = serve, args = (quit_serving,),  name = 'serving', daemon=True )

def stop_serving():
    global quit_serving
    global serving_thread
    quit_serving.set()
    if serving_thread.is_alive():
        logger.bind(serving=True).info("Joining...")
        serving_thread.join( timeout = join_timeout )
    if not serving_thread.is_alive():
        print ('Joined serving thread',)
        logger.bind(serving=True).info('Joined.')
    else:
        print ('Failed to join serving thread')

def start_serving():
    global quit_serving
    global serving_thread
    stop_serving()
    quit_serving = mp.Event()
    serving_thread = threading.Thread( target = serve, args = (quit_serving,), name = 'serving', daemon=True )
    serving_thread.start()
    logger.bind(serving=True).info("Started serving.")
    print('new serving thread:', serving_thread)


In [32]:
start_serving()

Joined serving thread
new serving thread: <Thread(serving, started daemon 123145873633280)>


In [33]:
print (serving_thread.is_alive())

True


In [60]:
stop_serving()

Joined serving thread


In [38]:
show_serving_logs(20)

2021-04-06 10:33:43.130 | INFO     | __main__:serve:22 - Recieved: [None, None, None, None]
2021-04-06 10:33:43.131 | INFO     | __main__:serve:20 - Axon:(0.000q/s|⤊ 0.0/⤋ 0.0kB/s), waiting for query ... 
2021-04-06 10:33:44.131 | INFO     | __main__:serve:22 - Recieved: [None, None, None, None]
2021-04-06 10:33:44.132 | INFO     | __main__:serve:20 - Axon:(0.000q/s|⤊ 0.0/⤋ 0.0kB/s), waiting for query ... 
2021-04-06 10:33:45.138 | INFO     | __main__:serve:22 - Recieved: [None, None, None, None]
2021-04-06 10:33:45.138 | INFO     | __main__:serve:20 - Axon:(0.000q/s|⤊ 0.0/⤋ 0.0kB/s), waiting for query ... 
2021-04-06 10:33:46.139 | INFO     | __main__:serve:22 - Recieved: [None, None, None, None]
2021-04-06 10:33:46.140 | INFO     | __main__:serve:20 - Axon:(0.000q/s|⤊ 0.0/⤋ 0.0kB/s), waiting for query ... 
2021-04-06 10:33:47.145 | INFO     | __main__:serve:22 - Recieved: [None, None, None, None]
2021-04-06 10:33:47.146 | INFO     | __main__:serve:20 - Axon:(0.000q/s|⤊ 0.0/⤋ 0.0kB/s)

## **Test-Serving** <a class="anchor" id="Test-Serving"></a>

In [37]:
endpoint = bittensor.proto.Neuron(
    address = axon.config.axon.external_ip,
    port = axon.config.axon.external_port,
    public_key = wallet.hotkey.public_key
)
start_time = time.time()
codes, responses = dendrite.forward_text( 
    neurons = [ endpoint ],
    inputs = [ torch.tensor([[1]]) ]
)
end_time = time.time()
print(colored('Querying endpoint: {}:{}'.format(axon.config.axon.external_ip, axon.config.axon.external_port), 'blue'))
if codes.item() == bittensor.proto.ReturnCode.Success:
    print(colored('Success', 'green'))
    print(colored('Response shape: {}'.format(responses[0].shape) , 'green'))
    print(colored('Query time: {}'.format(end_time - start_time) , 'green'))
else:
    print(colored('Failure with code: {}'.format(codes.item()), 'red'))
    print(colored('Ensure your axon is started with axon.start()', 'red'))
    print(colored('Ensure your endpoint is accessable from the internet, perhaps behind your router\'s NAT?', 'red'))


Querying endpoint: 172.58.142.212:8080
Failure with code: 1
Ensure your axon is started with axon.start()
Ensure your endpoint is accessable from the internet, perhaps behind your router's NAT?


# **Weights**

## **Filtering Weights**

In [69]:
# Get the trained weights from the chain.
weights_to_emit = torch.nn.functional.pad(
    model.row_weights, 
    pad = [0, metagraph.n() - model.row_weights.numel() ],
    value = torch.mean(model.row_weights).item()
)

model.row_weights.detach()

# Take topk
topk = 30
weights_to_emit, uids = torch.topk(weights_to_emit, topk)

# Normalize to 0,1
weights_to_emit = F.normalize(weights_to_emit, p = 1, dim = 0)
print ("Weights:\n{}\nFor uids\n {}".format(weights_to_emit.tolist(), uids.tolist()))



Weights:
[0.03333333507180214, 0.03333333507180214, 0.03333333507180214, 0.03333333507180214, 0.03333333507180214, 0.03333333507180214, 0.03333333507180214, 0.03333333507180214, 0.03333333507180214, 0.03333333507180214, 0.03333333507180214, 0.03333333507180214, 0.03333333507180214, 0.03333333507180214, 0.03333333507180214, 0.03333333507180214, 0.03333333507180214, 0.03333333507180214, 0.03333333507180214, 0.03333333507180214, 0.03333333507180214, 0.03333333507180214, 0.03333333507180214, 0.03333333507180214, 0.03333333507180214, 0.03333333507180214, 0.03333333507180214, 0.03333333507180214, 0.03333333507180214, 0.03333333507180214]
For uids
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]


## **Setting Weights**

In [70]:
# Sets your incentive weights on the chain.
subtensor.set_weights(
    uids = uids,
    weights = weights_to_emit
)

ValueError: not enough values to unpack (expected 3, got 2)

In [71]:
your_uid = bittensor.metagraph.uids()[0]
print ('Your weights (encoded as uint32s) on the chain are: \n\n {} \n'.format(bittensor.subtensor.weight_vals_for_uid( your_uid )))

print ('For uids \n {}'.format(bittensor.subtensor.weight_uids_for_uid( your_uid )))

AttributeError: module 'bittensor.metagraph' has no attribute 'uids'

# **Transactions** <a class="anchor" id="Transactions"></a>

## **Unstaking-Funds** <a class="anchor" id="Unstaking-Funds"></a>

In [122]:
from bittensor.utils.balance import Balance
amount_tao = 0.1
amount = Balance.from_float( amount_tao )
print(colored("Sending Extrinsic: [Unstake: {} Tao from hotkey: {}]".format( amount.tao, wallet.hotkey.public_key) , 'blue'))
print ('waiting for finalization...')
result = subtensor.unstake( amount, wallet.hotkey.public_key, wait_for_finalization = True, timeout = bittensor.__blocktime__ * 5)
if result:
    new_balance = subtensor.get_balance(wallet.coldkeypub)
    new_stake = subtensor.get_stake_for_uid( metagraph.uid_for_pubkey(wallet.hotkey.public_key) )
    print(colored("Unstaked: {} Tao from hotkey: {} to coldkey.pub: {}".format( amount.tao, wallet.hotkey.public_key, wallet.coldkey.public_key ) , 'green'))
    print(colored("Your coldkey has new balance: {} Tao".format( new_balance.tao ) , 'green'))
    print(colored("Your hotkey has new stake: {} Tao".format( new_stake.tao ) , 'green'))
else:
    print(colored("Unstaking transaction failed", 'red'))

Sending Extrinsic: [Unstake: 0.1 Tao from hotkey: 0x80cacfbdf7b155b39de22680a7cb14c61a8f95df702c92f5f142d25cca37c545]
waiting for finalization...
Successfully connected to kusanagi endpoint: 157.230.11.31:9944


Enter password to unlock key:  ··············


decrypting key... (this may take a few moments)
Invalid password


KeyError: Invalid password

## **Staking-Funds** <a class="anchor" id="Staking-Funds"></a>

In [123]:
from bittensor.utils.balance import Balance 
amount_tao = 0.1
amount = Balance.from_float( amount_tao )
print(colored("Sending Extrinsic: [Stake: {} Tao to hotkey: {}]".format( amount.tao, wallet.hotkey.public_key) , 'blue'))
print ('waiting for finalization...')
result = subtensor.add_stake( amount, wallet.hotkey.public_key, wait_for_finalization = True, timeout = bittensor.__blocktime__ * 5)
if result:
  new_balance = subtensor.get_balance(wallet.coldkeypub)
  new_stake = subtensor.get_stake_for_uid( metagraph.uid_for_pubkey(wallet.hotkey.public_key))
  print(colored("Staked: {} Tao to hotkey: {} from coldkey.pub: {}".format( amount.tao, wallet.hotkey.public_key, wallet.coldkey.public_key ) , 'green'))
  print(colored("Your coldkey has new balance: {} Tao".format( new_balance.tao ) , 'green'))
  print(colored("Your hotkey has new stake: {} Tao".format( new_stake.tao ) , 'green'))

else:
  print(colored("Staking transaction failed", 'red'))

Sending Extrinsic: [Stake: 0.1 Tao to hotkey: 0x80cacfbdf7b155b39de22680a7cb14c61a8f95df702c92f5f142d25cca37c545]
waiting for finalization...


Enter password to unlock key:  ··············


decrypting key... (this may take a few moments)
Invalid password


KeyError: Invalid password

## **Transfering-Funds** <a class="anchor" id="Transfering-Funds"></a>

In [201]:
amount = 0.01
destination_public_key = wallet.coldkey.public_key
amount = Balance.from_float( amount )
balance = bittensor.subtensor.get_balance( wallet.coldkey.public_key )
if balance < amount:
    print(colored("Not enough balance ({}) to transfer {}".format(balance, amount), 'red'))
    quit()

print(colored("Requesting transfer of {} Tao, from coldkey.pub: {} to dest.pub: {}".format(amount.tao, wallet.coldkey.public_key, destination_public_key), 'blue'))
print("Waiting for finalization...",)
result = bittensor.subtensor.transfer(destination_public_key, amount, wait_for_finalization = True, timeout = bittensor.__blocktime__ * 5)
if result:
    print(colored("Transfer finalized with amount: {} Tao to dest: {} from coldkey.pub: {}".format(amount.tao, destination_public_key, wallet.coldkey.public_key), 'green'))
    new_balance = bittensor.subtensor.get_balance(wallet.coldkeypub)
    destination_balance = bittensor.subtensor.get_balance(destination_public_key)
    print(colored("Your coldkey has new balance: {} Tao".format( new_balance.tao ) , 'green'))
    print(colored("The destination has new balance: {} Tao".format( new_balance.tao ) , 'green'))
else:
    print(colored("Transfer failed", 'red'))

AttributeError: module 'bittensor' has no attribute 'utils'